# ---> Using the model in only 3 steps <---

#Step 01 : Installation of necessary libraries

In [ ]:
!pip install --upgrade pip
!pip install langchain
!pip install sentencepiece

!pip uninstall transformers -y
!pip install transformers==4.36.0
!pip uninstall tokenizers -y
!pip install tokenizers==0.15.0
!pip uninstall peft -y
!pip install peft==0.7.0
!pip install bitsandbytes
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.6 MB/s eta 0:00:00
Found existing i

#Step 02 : Restart Session

## You can do this as follows : Runtime > Restart session

#Step 03 : Load model

### import libraries

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM , pipeline, LlamaConfig,AutoModel,AutoModelForCausalLM
from peft import PeftModel
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import pandas as pd
from tqdm import tqdm
import torch
import warnings
warnings.filterwarnings("ignore")

## import tokenizer

In [ ]:
path = 'mostafaamiri/persian_llama_7b'

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(path)

tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/938k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

## load base model

In [ ]:
base_model = LlamaForCausalLM.from_pretrained(
    "mostafaamiri/base_7B",
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto",
    )

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
base_model.resize_token_embeddings(len(tokenizer))

Embedding(51954, 4096)

In [ ]:
model = PeftModel.from_pretrained(
    base_model,
    path,
    device_map='auto'
)

adapter_config.json:   0%|          | 0.00/417 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

## Model Function

In [ ]:
def UseModel(input):
  prompt = input

  inputToken = tokenizer(prompt, return_tensors="pt")
  config=dict(
      temperature=0.2,
      top_k=40,
      top_p=0.9,
      do_sample=True,
      num_beams=1,
      repetition_penalty=1.2,
      max_new_tokens=200
      )
  outputs = model.generate(**inputToken, **config)
  output = tokenizer.decode(outputs[0],skip_special_tokens=True)

  print(output)

#Finally : Use Model
### For this, ask your question to the 'UseModel' function.

### for example :
`UseModel('یک داستان کوتاه بهم بگو.')`

In [ ]:
UseModel("پایتخت ایران در طول تاریخ چه شهرهایی بوده است؟")

پایتخت ایران در طول تاریخ چه شهرهایی بوده است؟ پایتخت ایران از زمان باستان تا به امروز چندین بار تغییر کرده است. این شهر ها عبارتند از : 1. بابل (حدود سال 2500 پیش از میلاد) ، 2. پرسپولیس (حدود سال 648 میلادی) ، 3. اصفهان (حدود سال 796 میلادی) ، 4- شیراز (حدود سال 1168 میلادی) ، 5. تبریز (حدود سال 1260 میلادی) ، 6. قزوین (حدود سال 1368 میلادی) ، 7. تهران (حدود سال 1310 میلادی). پایتخت ایران در طول تاریخ بارها و بارها تغییر کرده است زیرا امپراتوری های مختلف بر قلمرو حکومت می کردند.
